**Zero-Shot Prompting**

	• What it is: You give the model only the instructions, context, and question. No examples.
	• When to use:
	• When you don’t have sample Q&A pairs.
	• For general-purpose answering.
	• When you want the model to reason freely but still stay grounded in the context.
	• Trade-off: Faster and simpler, but sometimes answers may not follow the exact style or format you want.

Example:

Context about Transformers → Ask: “How do they handle long-range dependencies?”

The model figures out the answer directly from context.

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.groq import Groq

c:\Users\user\Desktop\Gen_AI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def run_llm(context:str , query:str)-> str:
    llm = Groq(
        model = "llama-3.1-8b-instant",
        temperature=0
    )

    template_str = (
        "You are an expert AI assistant.\n"
        "Use ONLY use the provided context to answer the user's question. "
        "If the context is insufficient or does not mention the answer, reply exactly: "
        "'Not enough information.'\n\n"
        "Context:\n{context_str}\n\n"
        "User Question: {query_str}\n\n"
        "Answering Rules:\n"
        "1) Be concise and precise (3–6 sentences, unless the question requires more).\n"
        "2) Use bullet points for lists.\n"
        "3) At the end, include a 'Sources:' section with short snippets or filenames from the context you used.\n\n"
        "Final Answer:"
    )

    prompt = PromptTemplate(template_str)
    filled_prompt = prompt.format(
        context_str = context,
        query_str = query
    )
    response = llm.complete(filled_prompt)
    output = response.text
    return output

In [3]:
# Zero-Shot: No examples, just context + query
context_text = (
    "Transformers use a self-attention mechanism that allows each token "
    "to attend to all other tokens in the sequence. This helps capture "
    "long-range dependencies without recurrence."
)

query_text = "How do Transformers handle long-range dependencies?"

ans0 = run_llm(context=context_text, query=query_text)

print(ans0)

Transformers handle long-range dependencies using a self-attention mechanism. This mechanism allows each token to attend to all other tokens in the sequence, enabling the model to capture long-range relationships.

Key features of the self-attention mechanism:

* Each token attends to all other tokens in the sequence.
* This allows the model to capture long-range dependencies without recurrence.
* The attention weights are calculated based on the similarity between tokens.

Sources:
Transformers use a self-attention mechanism that allows each token to attend to all other tokens in the sequence. This helps capture long-range dependencies without recurrence.


**Few-Shot Prompting**

	• What it is: You give the model a few examples of context → question → answer, then add your new context + question.
	• When to use:
	• When you want the model to mimic a specific style (tone, structure, format).
	• If you want consistent answers (e.g., always ending with “Sources: …”).
	• Good for teaching the model special formatting rules or domain-specific style.
	• Trade-off: More control, but you must prepare good examples, and the prompt can get longer.

Example:

Show 2–3 sample Q&A pairs → Then ask a new question with new context.
The model copies the answering style from your examples.

In [4]:

from typing import List, Dict

def run_llm_fewshot(context: str, query: str, examples: List[Dict[str, str]]) -> str:
    llm = Groq(model="llama-3.3-70b-versatile", temperature=0)

    examples_str = "\n".join(
        f"Example {i+1}:\nContext:\n{ex.get('context','')}\n"
        f"Question: {ex.get('question','')}\n"
        f"Answer: {ex.get('answer','')}\n"
        for i, ex in enumerate(examples)
    )

    template_str = (
        "You are an expert AI assistant.\n"
        "Use ONLY the provided context to answer the user's question. "
        "If the context is insufficient or does not mention the answer, reply exactly: "
        "'Not enough information.'\n\n"
        "Follow the style and reasoning illustrated by the examples.\n\n"
        "Examples:\n{examples_str}\n"
        "--- End of Examples ---\n\n"
        "Context:\n{context_str}\n\n"
        "User Question: {query_str}\n\n"
        "Answering Rules:\n"
        "1) Be concise and precise (3–6 sentences, unless the question requires more).\n"
        "2) Use bullet points for lists.\n"
        "3) At the end, include a 'Sources:' section with short snippets or filenames from the context you used.\n\n"
        "Final Answer:"
    )
    prompt = PromptTemplate(template_str).format(
        examples_str=examples_str, context_str=context, query_str=query
    )
    return llm.complete(prompt=prompt).text

In [5]:
shots = [
    {
        "context": "Positional encodings inject order information into sequences.",
        "question": "Why are positional encodings needed?",
        "answer": (
            "They give the model a sense of word order.\n"
            "- Without them, the model treats tokens as a bag of words.\n"
            "- Encodings ensure the sequence structure is preserved.\n"
            "Sources: lecture_notes.txt"
        )
    },
    {
        "context": "Multi-head attention projects queries, keys, and values into multiple subspaces.",
        "question": "What is the benefit of multi-head attention?",
        "answer": (
            "It lets the model learn from different representation subspaces.\n"
            "- Captures diverse relationships.\n"
            "- Improves contextual understanding.\n"
            "Sources: attention_paper.pdf"
        )
    },
]

context_text = (
    "Context from attention_mechanism.pdf"
    "In the attention mechanism, softmax is used on the similarity scores "
    "between queries and keys to produce attention weights."
)

query_text = "what does softmax do in attention"

answer = run_llm_fewshot(
    context = context_text,
    query = query_text,
    examples = shots
)

print(answer)

In the attention mechanism, softmax is used to normalize the similarity scores between queries and keys. This process produces attention weights that represent the relative importance of each key with respect to the query. The softmax function ensures that the attention weights add up to 1, allowing the model to focus on the most relevant keys. Key benefits of using softmax include:
* Normalizing the similarity scores
* Producing a probability distribution over the keys
Sources: attention_mechanism.pdf


**👉 Rule of thumb:**

	• Use Zero-Shot for quick, flexible answers.
	• Use Few-Shot when consistency, formatting, or special style matters.